# Linear Programming Problem Set

This notebook contains a collection of Linear Programming (LP) and Integer Linear Programming (ILP) modeling problems.  
Each section presents one problem description that can be later formulated and solved using optimization tools in Python.

In [ ]:
# installing and importing the mip package
!pip install mip

from mip import *

In [ ]:
def solve(model):
  status = model.optimize()

  print("Status = ", status)
  if status != OptimizationStatus.OPTIMAL:
    return

  print(f"Solution value  = {model.objective_value:.2f}\n")

  print("Solution:")
  for v in model.vars:
    if v.x > 0.001:
      print(f"{v.name} = {v.x:.2f}")

In [ ]:
def save(model, filename):
  model.write(filename)
  with open(filename, "r") as f:
    print(f.read())